# comercio_ext_auxiliares.tb_auxiliar_recintos_alfandegarios
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/URF_delta`
## 📌 Descrição do arquivo
Referência de **URF** (recintos alfandegários).
|Coluna|Descrição|
|---|---|
|`CO_URF`|Código URF|
|`NO_URF`|Nome URF|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:
val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/URF_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_auxiliares/tb_auxiliar_recintos_alfandegarios/"   
val silverTable =    "tb_auxiliar_recintos_alfandegarios"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)

In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_URF" , StringType ,  nullable = false ) ,
      StructField ( "NO_URF" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_URF", trim(upper(col("CO_URF").cast(StringType))))
  .withColumn("NO_URF", col("NO_URF").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/URF_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_URF").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_URF"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_URF"), col("NO_URF"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}